<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Colaboratoryで実行する場合のみ実行" data-toc-modified-id="Colaboratoryで実行する場合のみ実行-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Colaboratoryで実行する場合のみ実行</a></span></li><li><span><a href="#前処理" data-toc-modified-id="前処理-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>前処理</a></span></li><li><span><a href="#ニューラルネットワークの訓練" data-toc-modified-id="ニューラルネットワークの訓練-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>ニューラルネットワークの訓練</a></span></li><li><span><a href="#ニューラルネットワークの評価" data-toc-modified-id="ニューラルネットワークの評価-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>ニューラルネットワークの評価</a></span><ul class="toc-item"><li><span><a href="#学習の進捗" data-toc-modified-id="学習の進捗-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>学習の進捗</a></span></li><li><span><a href="#データ量とテストデータでの評価項目の関連" data-toc-modified-id="データ量とテストデータでの評価項目の関連-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>データ量とテストデータでの評価項目の関連</a></span></li><li><span><a href="#ROC曲線" data-toc-modified-id="ROC曲線-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>ROC曲線</a></span></li><li><span><a href="#Grad-CAM" data-toc-modified-id="Grad-CAM-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Grad-CAM</a></span></li></ul></li></ul></div>

# Colaboratoryで実行する場合のみ実行
- [Googleドライブの読み込みと保存](https://colab.research.google.com/notebooks/io.ipynb)

In [ ]:
import glob
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')

!mkdir images
!mkdir logs

!cp gdrive/My\ Drive/x-ray/* .
!cp gdrive/My\ Drive/x-ray/images/* images

In [ ]:
gz_list = glob.glob("images/*.tar.gz")

for gz_file in gz_list:
    tar = tarfile.open(gz_file)
    tar.extractall()
    tar.close()

!pip install Keras==2.2.4
!pip install Keras-Applications==1.0.6
!pip install Keras-Preprocessing==1.0.5

# 前処理

In [ ]:
import os
import glob
import shutil
import numpy as np
import pandas as pd
import random
import pathlib
import keras
from keras_preprocessing.image import (array_to_img, img_to_array, load_img)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from IPython.display import display

#auto relord modules
%load_ext autoreload
%autoreload 2

In [ ]:
random.seed(2018)
df = pd.read_csv('Data_Entry_2017.csv', encoding='us-ascii')

# Keep df where the file exist in images folder.
file_list = glob.glob("images/*")
file_list = [os.path.basename(file) for file in file_list]
file_list = pd.DataFrame(file_list, columns=['Image Index'])
df = pd.merge(df, file_list, on='Image Index', how='inner')

Findings = df["Finding Labels"].str.split('|')
Findings_list = [word for word_inner in Findings for word in word_inner]
Findings_list = pd.DataFrame(Findings_list, columns=['text'])
print(Findings_list['text'].value_counts())
Findings_list = list(Findings_list.drop_duplicates()['text'])
Findings_list.remove('No Finding')
for finding in Findings_list:
    df[finding] = Findings.apply(lambda x: 1 if finding in x else 0)

In [ ]:
# Create train_list.txt and validation_list.txt

random.seed(2018)
train_val_list = set([])
with open('train_val_list.txt', 'r') as f:
    for row in f:
        train_val_list.add(row.rsplit()[0])
train_list = random.sample(train_val_list, int(0.8 * len(train_val_list)))
train_list = sorted(train_list)
validation_list = train_val_list - set(train_list)
validation_list = sorted(validation_list)
with open('train_list.txt', 'w') as f:
    f.write("\n".join(train_list))
with open('validation_list.txt', 'w') as f:
    f.write("\n".join(validation_list))

In [ ]:
for tt in ['train', 'validation', 'test']:
    tmp_list = []
    with open(tt + '_list.txt', 'r') as f:
        for row in f:
            tmp_list.append(row.rsplit()[0])
    df[tt] = df['Image Index'].isin(tmp_list).apply(
        lambda x: 'Y' if x == True else 'N')


def data_split(x):
    if x.test == "Y":
        return "test"
    elif x.train == "Y":
        return "train"
    else:
        return "validation"


df['split'] = df.apply(lambda x: data_split(x), axis=1)

# ニューラルネットワークの訓練

In [ ]:
import myTrainer

y_col = ['Effusion']

In [ ]:
# trainingx = myTrainer.trainer(
#     trainer_name='densenet_p3',
#     dataframe=df,
#     data_proportion=10**-2,
#     y_col=y_col,
#     lr=10**-7,
#     batch_size=32,
#     initial_epoch=0,
#     epochs=50)
# trainingx.lr_finder('densenet_p3')

# trainingx = myTrainer.trainer(
#     trainer_name='densenet_p2',
#     dataframe=df,
#     data_proportion=10**-2,
#     y_col=y_col,
#     lr=10**-7,
#     batch_size=32,
#     initial_epoch=0,
#     epochs=50)
# trainingx.lr_finder('densenet_p2')

# trainingx = myTrainer.trainer(
#     trainer_name='densenet_p1',
#     dataframe=df,
#     data_proportion=10**-1,
#     y_col=y_col,
#     lr=10**-7,
#     batch_size=32,
#     initial_epoch=0,
#     epochs=50)
# trainingx.lr_finder('p1')

# trainingx = myTrainer.trainer(
#     trainer_name='densenet_p0',
#     dataframe=df,
#     data_proportion=10**-0,
#     y_col=y_col,
#     lr=10**-7,
#     batch_size=32,
#     initial_epoch=0,
#     epochs=50)
# trainingx.lr_finder('densenet_p0')

In [ ]:
# import myTrainer
# training1 = myTrainer.trainer(
#     trainer_name='densenet_lr7_p3',
#     dataframe=df,
#     data_proportion=10**-3,
#     y_col=y_col,
#     lr=10**-7,
#     batch_size=32,
#     initial_epoch=0,
#     epochs=10)
# training1.training()
# training1.resume_training('densenet_lr7_p3_1')

In [ ]:
training3 = myTrainer.trainer(
    trainer_name='densenet_lr3_p3',
    dataframe=df,
    data_proportion=10**-3,
    y_col=y_col,
    lr=10**-3,
    batch_size=32,
    initial_epoch=0,
    epochs=50)
training3.training()
training3.evaluating()

In [ ]:
training2 = myTrainer.trainer(
    trainer_name='densenet_lr3_p2',
    dataframe=df,
    data_proportion=10**-2,
    y_col=y_col,
    lr=10**-3,
    batch_size=32,
    initial_epoch=0,
    epochs=50)
training2.training()
training2.evaluating()

In [ ]:
training1 = myTrainer.trainer(
    trainer_name='densenet_lr3_p1',
    dataframe=df,
    data_proportion=10**-1,
    y_col=y_col,
    lr=10**-3,
    batch_size=32,
    initial_epoch=0,
    epochs=50)
training1.training()
training1.evaluating()

In [ ]:
training0 = myTrainer.trainer(
    trainer_name='densenet_lr3_p0',
    dataframe=df,
    data_proportion=10**-0,
    y_col=y_col,
    lr=10**-3,
    batch_size=32,
    initial_epoch=0,
    epochs=50)
training0.training()
training0.evaluating()

# ニューラルネットワークの評価
## 学習の進捗

In [ ]:
df_training_log = pd.DataFrame()
for _ in range(4):
    tmp = pd.read_csv(
        './logs/training_log_densenet_lr3_p' + str(_) + '.csv', sep='\t')
    tmp['power'] = _
    df_training_log = pd.concat([df_training_log, tmp])

vars = ['acc', 'loss', 'auc', 'fscore', 'precision', 'recall']
epoch = df_training_log['epoch'].unique() + 1
sns.set()
cmap = plt.get_cmap("Blues")
for var in vars:
    for power in range(4):
        training = df_training_log.query('power==' + str(power))[var]
        val = df_training_log.query('power==' + str(power))['val_' + var]
        plt.plot(
            epoch,
            training,
            'bo',
            label=str(10**-power) + ': Training ' + var,
            color=cmap((4 - power) / 4))
        plt.plot(
            epoch,
            val,
            'b',
            label=str(10**-power) + ': Validation ' + var,
            color=cmap((4 - power) / 4))
    plt.title('Training and validation ' + var)
    plt.legend()
    plt.savefig("logs/Effusion_Train_Val_p0-p3_" + var + ".svg")
    plt.figure()

plt.show()

In [ ]:
for var in vars:
    for power in [0]:
        training = df_training_log.query('power==' + str(power))[var]
        val = df_training_log.query('power==' + str(power))['val_' + var]
        plt.plot(
            epoch,
            training,
            'bo',
            label='Training ' + var,
            color=cmap((4 - power) / 4))
        plt.plot(
            epoch,
            val,
            'b',
            label='Validation ' + var,
            color=cmap((4 - power) / 4))
    plt.title('Training and validation ' + var)
    plt.legend()
    plt.savefig("logs/Effusion_Train_Val_p0_" + var + ".svg")
    plt.figure()

plt.show()

## データ量とテストデータでの評価項目の関連

In [ ]:
df_test_log = pd.DataFrame()
for _ in range(4):
    tmp = pd.read_csv(
        './logs/test_result_densenet_lr3_p' + str(_) + '.csv', sep='\t')
    tmp['power'] = _
    tmp['frac'] = 10**(-_)
    df_test_log = pd.concat([df_test_log, tmp])

vars = ['acc', 'loss', 'auc', 'fscore', 'precision', 'recall']
sns.set()
cmap = plt.get_cmap("Blues")
#cmap = sns.cubehelix_palette(4, as_cmap=True)
for var in vars:
    y = df_test_log[var]
    x = df_test_log['frac']
    plt.plot(x, y, 'b', color=cmap(1.0))
    plt.title('Test ' + var)
    plt.legend()
    plt.xscale("log")
    plt.savefig("logs/Effusion_Test_" + var + ".svg")
    plt.figure()

plt.show()

## ROC曲線

In [ ]:
#http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
roc_auc_list, fpr_list, tpr_list = training0.auc_test()
plt.figure()
for i, finding in enumerate(y_col):
    fpr = fpr_list[i]
    tpr = tpr_list[i]
    roc_auc = roc_auc_list[i]
    plt.plot(
        fpr,
        tpr,
        #color='darkorange',
        lw=2,
        label=finding + ' (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig("logs/Effusion_Test_ROC.svg")
plt.show()

## Grad-CAM

In [ ]:
training0.Grad_CAM('00021381_013.png')

In [ ]:
training0.Grad_CAM('00001437_012.png')
training0.Grad_CAM('00001558_016.png')
training0.Grad_CAM('00029894_000.png')
training0.Grad_CAM('00013337_000.png')
training0.Grad_CAM('00021181_002.png')
training0.Grad_CAM('00012045_009.png')
'''
00002395_007.png
00027028_017.png
00010007_168.png
00028974_016.png
00016291_002.png
00023058_004.png
00020277_001.png
00030634_000.png
00012834_034.png
00018427_011.png
00007034_016.png
00012834_122.png
00016972_025.png
00023283_019.png
00013285_026.png
00017714_006.png
00027631_000.png
00020751_003.png
'''